In [1]:
import numpy as np

In [10]:
# Função para gerar a função target
def generate_target_function():
    points = np.random.uniform(-1, 1, (2, 2))
    a = points[1, 1] - points[0, 1]
    b = points[0, 0] - points[1, 0]
    c = points[1, 0] * points[0, 1] - points[0, 0] * points[1, 1]
    return a, b, c

# Função target
def target_function(a, b, c, x):
    return np.sign(a * x[:, 0] + b * x[:, 1] + c)

# Gerar dados
def generate_data(N, a, b, c):
    X = np.random.uniform(-1, 1, (N, 2))
    y = target_function(a, b, c, X)
    return X, y


# Regressão Linear
def linear_regression(X, y):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Adicionar x0 = 1 para cada instância
    w = np.linalg.pinv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)  # w = (X_b^T * X_b)^-1 * X_b^T * y
    return w


In [16]:
# Calcular E_in
def calculate_error(X, y, w):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    predictions = np.sign(X_b.dot(w))
    return np.mean(predictions != y)


In [27]:
# PLA
def pla(X, y, w):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    iterations = 0
    while True:
        predictions = np.sign(X_b.dot(w))
        misclassified = np.where(predictions != y)[0]
        if len(misclassified) == 0:
            break
        idx = np.random.choice(misclassified)
        w += y[idx] * X_b[idx]
        iterations += 1
    return iterations

In [29]:
# Experimento
def experiment_rg(num_runs, num_points_train, num_points_test):
    ein_list = []
    eout_list = []

    for _ in range(num_runs):
        a, b, c = generate_target_function()
        X_train, y_train = generate_data(num_points_train, a, b, c)
        w = linear_regression(X_train, y_train)
        ein = calculate_error(X_train, y_train, w)
        ein_list.append(ein)

        # Gerar novos pontos de teste
        X_test, y_test = generate_data(num_points_test, a, b, c)
        eout = calculate_error(X_test, y_test, w)
        eout_list.append(eout)

    return np.mean(ein_list), np.std(ein_list), np.mean(eout_list), np.std(eout_list)


# Experimento
def experiment_rg_pla(num_runs, num_points_train):
    iterations_list = []

    for _ in range(num_runs):
        a, b, c = generate_target_function()
        X_train, y_train = generate_data(num_points_train, a, b, c)
        w = linear_regression(X_train, y_train)
        iterations = pla(X_train, y_train, w)
        iterations_list.append(iterations)

    return np.mean(iterations_list), np.std(iterations_list)

In [20]:
# Parâmetros
num_runs = 1000
num_points_train = 100
num_points_test = 1000

# Executar o experimento
mean_ein, std_ein, mean_eout, std_eout = experiment_rg(num_runs, num_points_train, num_points_test)
print(f"Média de E_in: {mean_ein}")
print(f"Desvio padrão de E_in: {std_ein}")
print(f"Média de E_out: {mean_eout}")
print(f"Desvio padrão de E_out: {std_eout}")

Média de E_in: 0.039540000000000006
Desvio padrão de E_in: 0.030799162326271147
Média de E_out: 0.04873
Desvio padrão de E_out: 0.03260860469262676


In [31]:
# Parâmetros
num_runs = 1000
num_points_train = 10

# Executar o experimento
mean_iterations, std_iterations = experiment_rg_pla(num_runs, num_points_train)
print(f"Média de iterações até a convergência do PLA: {mean_iterations}")
print(f"Desvio padrão de iterações: {std_iterations}")

Média de iterações até a convergência do PLA: 3.587
Desvio padrão de iterações: 11.37050706872829
